In [ ]:
from lxml import etree

from zipfile import ZipFile
from pathlib import Path

from io import BytesIO

from ooxml_ns import *

In [ ]:
from filenameparser import FileNameParser

In [ ]:
file = r"..\input\45-Day\100-MDAQMD2.docx"

In [ ]:
class Document:
    def __init__(self, filename):
        self.file = FileNameParser(filename)

    @property
    def xml(self):
        with ZipFile(self.file.path, "r") as z:
            return {
                filename: etree.iterparse(BytesIO(z.read(filename)), 
                                        #   events=["start","end"],
                                          tag=["{*}commentRangeStart", "{*}p", "{*}r", "{*}commentRangeEnd"])
                for filename in z.namelist()
                if ".xml" in filename
            }

In [ ]:
from document import Document

In [ ]:
doc = Document(file)

In [ ]:
def things(doc):
    for event, element in doc.xml['word/document.xml']:
        if "commentRangeStart" in element.tag:
            _id = element.attrib[f"{{{ooXMLns['w']}}}id"]
            yield element
        if "commentRangeEnd" in element.tag and element.attrib[f"{{{ooXMLns['w']}}}id"] == _id:
            raise StopIteration

In [ ]:
for element in doc.xml['word/document.xml'][0]:
    print(element.tag)

In [ ]:
gen = things(doc)

In [ ]:
for g in gen:
    for element in g.iter():
        print(element)